# Generate Lowpass Filtered Pink Noise

with the Voss-McCartney algorithm
using implementation of 

https://github.com/AllenDowney/ThinkDSP/blob/master/code/voss.ipynb

see also

http://www.firstpr.com.au/dsp/pink-noise/

In [1]:
import numpy as np
import soundfile as sf
import util
from scipy.signal import fftconvolve as conv, butter, lfilter

In [2]:
fs = 44100  # Hz
t = 6*60  # s
lpf_order = 4
f_cutoff = 300  # Hz

np.random.seed(1024)
src_dir = '../data/source-signals'
suffix = '.wav'

b, a = butter(lpf_order, Wn=f_cutoff, btype='lowpass', fs=fs)
pn = util.pink_noise(int(t*fs))
pn = pn - np.mean(pn)
pn = lfilter(b, a, pn)
pn = pn - np.mean(pn)
pn = pn / np.max(np.abs(pn)) * 0.25  # peak -12 dB
sf.write(src_dir+'/pnoise_voss_full'+suffix,pn,fs, subtype='PCM_24')

## Pink Noise for ABX Training and SPL Metering with Ear Coupler

In [3]:
fs = 44100
fade_in, fade_out = 32, 32
t = 3  # s
np.random.seed(1024)
src_dir = '../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli'
suffix = '.wav'
x = util.pink_noise(int(t*fs))
x = x - np.mean(x)
x = x / np.max(np.abs(x)) * 0.25  # peak -12 dB
x1 = util.fade(x, fade_in, fade_out)
sf.write(src_dir+'/pnoise_ref'+suffix,np.column_stack((x1, x1)),fs, subtype='PCM_24')
sf.write(src_dir+'/pnoise_ref_mono'+suffix,x1,fs, subtype='PCM_24')
# signal statistics, these results are comparable with the TBProAudio dpMeter3 plugin
print('sample peak = ', 20*np.log10(np.max(np.abs(x))), 'dB')
print('true peak = ', 20*np.log10(np.max(np.abs(util.resample(x, 16 * len(x))))), 'dB')
print('RMS = ', 10*np.log10(np.mean(x1**2)),'dB')
print('loudness = ', util.get_LUFS_Stereo(x1,fs, mono_gated=False), 'LUFS (R128, ITU BS.1770/4)')
print('crest factor = ', 20*np.log10(util.crest_factor(x, oversample=16)), 'dB')
x1 = util.fade(x*10**(-1/20), fade_in, fade_out)  # lower level 
sf.write(src_dir+'/pnoise_treat'+suffix,np.column_stack((x1, x1)),fs, subtype='PCM_24')

sample peak =  -12.041199826559248 dB
true peak =  -11.648264978425868 dB
RMS =  -25.148887818972923 dB
loudness =  -26.889581769829274 LUFS (R128, ITU BS.1770/4)
crest factor =  13.500134135814019 dB
